# Chapter 6: Building Real Taproot Contracts> **Reference**: `book/manuscript/Chapter 6 Building Real Taproot Contracts - Single-Leaf Hash Lock and Dual-Path Spending.md`  > **Code Examples**: `code/chapter06/`  > **Last Updated**: 2025-12-06---## Why Script Path Changes Everything

What if you could build a contract on Bitcoin that looks like a simple payment—until it reveals a puzzle that unlocks funds?

Imagine this: you want to create a digital bounty where anyone who solves a puzzle gets rewarded, but if no one solves it, you want to reclaim your funds. Or you're selling digital goods where buyers automatically get unlock keys after payment, but you retain refund control.

Traditional Bitcoin scripts either completely expose all conditions (compromising privacy) or require complex multi-signature setups (inefficient). Worse yet, even simple single-signature payments can be easily identified by their transaction patterns on the blockchain.

**Taproot's Script Path makes all this elegant**: complex conditional contracts look identical to ordinary payments when unspent, revealing only the necessary execution path when needed.

Building on the Taproot theoretical foundation established in previous chapters, we'll now dive deep into Script Path spending patterns. This chapter demonstrates how to implement dual-path authorization in Taproot's script tree through a practical conditional payment scenario: supporting both secret-based conditional payments and direct key holder control.

Let's start with a concrete business scenario to understand the practical value of Taproot Script Path:Alice wants to create a conditional payment contract with these characteristics:- **Conditional Path**: Anyone who knows the secret word "helloworld" can claim the funds- **Alternative Path**: Alice, as the fund owner, can reclaim her funds at any time using her private key- **Privacy Requirement**: When unspent, external observers cannot distinguish between simple payments and complex contractsThis dual-path design is extremely common in real applications:| **Application Scenario** | **Description** ||-------------------------|----------------|| **Digital Goods Sales** | Buyer gets unlock keys after payment; seller retains refund authority || **Bounty Tasks** | Puzzle solvers claim reward; unused bounty can be reclaimed by publisher || **Conditional Escrow** | Automatically releases under specific conditions; owner can reclaim otherwise || **Educational Incentives** | Students claim reward upon correct answers; teachers retain management control |

In our scenario, Alice's Taproot address contains two spending paths:**Key Path (Collaborative Path)**:- Alice signs directly with her tweaked private key- 64-byte Schnorr signature, maximum efficiency- Complete privacy, no script information leakage**Script Path (Script Path)**:- Uses Hash Lock script: `OP_SHA256 <hash> OP_EQUALVERIFY OP_TRUE`- Anyone who knows the preimage "helloworld" can spend- Requires revealing script content, but doesn't expose Key Path existence

Before diving into code implementation, we need to understand Taproot's core development pattern: **Commit-Reveal**. This pattern will become the fundamental development model for all our subsequent Taproot applications.### Commit-Reveal Pattern Overview**Commit Phase**:- Build script tree containing multiple spending conditions- Commit script tree to a Taproot address- Generate "intermediate address" or "custody address" to lock funds- **External parties cannot know** specific spending conditions**Reveal Phase**:- Choose one spending condition for unlocking- Key Path: Spend directly with key (maximum privacy)- Script Path: Reveal and execute specific script branch- **Only expose actually used conditions**, other conditions remain private foreverThe power of this pattern lies in: During Commit phase, all contracts of different complexity look identical; During Reveal phase, only the actually used branch needs to be exposed.

Let's learn the complete implementation flow of Taproot single leaf scripts through a simple Hash Lock case. Based on Alice's conditional payment scenario introduced earlier, we'll implement:- **Hash Lock Script**: Verify hash value of secret word "helloworld"- **Single Leaf Structure**: Simplest script tree containing only one script branch- **Dual Path Spending**: Key Path (Alice's direct control) + Script Path (conditional spending)### What is Tagged Hash?Before entering specific implementation, we need to understand an important concept: **Tagged Hash**. This is a hashing method introduced by BIP340 that adds tag prefixes for different purposes:This design prevents hash collisions between different protocols, greatly enhancing security. Each tag has its specific purpose, ensuring hash values from different contexts never accidentally duplicate.### Phase 1: Commit Phase - Build Intermediate Address to Lock FundsFirst, Alice needs to commit her Hash Lock script to a Taproot address:**Key Technical Points Analysis:**1. **Script Serialization**:    ```    a820936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af8851    ```    - `a8`: OP_SHA256    - `20`: PUSH 32 bytes    - `936a185c...07af`: SHA256("helloworld")    - `88`: OP_EQUALVERIFY    - `51`: OP_TRUE2. **TapLeaf Hash Calculation**:    3. **Output Key Generation**:    **Generated Intermediate Address**: `tb1p53ncq9ytax924ps66z6al3wfhy6a29w8h6xfu27xem06t98zkmvsakd43h`This is our **intermediate address** or **custody address** where funds are locked. The corresponding ScriptPubKey: `OP_1 <32-byte-output-key>`, is completely indistinguishable from any other Taproot address on the blockchain. External observers cannot distinguish whether this is a simple single-signature or complex conditional contract.### Phase 2: Reveal Phase - Key Path Spending (Alice's Direct Control)If Alice decides to directly reclaim her funds, she can use Key Path spending:**Key Path Characteristics Analysis**:- **Witness Size**: 64 bytes (fixed-size Schnorr signature)- **Privacy Level**: Complete privacy, no script information leakage- **Execution Efficiency**: Highest efficiency, single signature verification- **Appearance**: Identical to any simple Taproot payment**Important Technical Detail**: Even when using Key Path, the signing function still needs the `tapleaf_scripts` parameter to calculate the correct tweak value. This is because Taproot's output key is generated through both internal public key and script tree.**Key Path Signing Principle**: Alice needs to sign with the tweak-adjusted private key to unlock funds. Essentially, this is also restoring the intermediate address process. Although the `script_path=False` parameter lets the library hide these details, the underlying principle is:- **Public Key Linear Correspondence**: `output_pubkey = internal_pubkey + tweak * G`- **Private Key Linear Correspondence**: `tweak_private = internal_private + tweak`- **Schnorr Signature Property**: This linear relationship ensures key pair consistencyThis is exactly the advantage of Schnorr signatures over ECDSA: supporting linear combination and aggregation of keys.### Phase 3: Reveal Phase - Script Path Spending (Conditional Unlock)Now let's implement the complete code for Script Path spending. Unlike Key Path's simplicity, Script Path requires building more complex witness data to prove we have the right to use a specific script branch.### Script Path Spending Code Implementation### Detailed Key Implementation AnalysisLet's dive deep into the implementation principles of each key component:**1. Control Block Construction****Control Block Structure Diagram**:```Control Block Structure (33 bytes):┌─────────┬──────────────────────────────────┐│ Byte 1  │           Bytes 2-33             │├─────────┼──────────────────────────────────┤│   c1    │ 50be5fc4...126bb4d3             │├─────────┼──────────────────────────────────┤│Ver+Parity│         Internal Pubkey          │└─────────┴──────────────────────────────────┘Analysis:- c1 = c0 (leaf version) + 01 (parity flag)- Internal pubkey: Used to recalculate output key during verification```**Technical Points**:- **Internal Public Key**: This is the original public key before tweak adjustment, used to recalculate output key during verification- **Script Tree Structure**: `[[tr_script]]` represents a script tree with only one leaf, multiple scripts would be `[[script1], [script2]]`- **Script Index**: Identifies which script to use in multi-script tree, always 0 for single script- **Parity Flag**: Elliptic curve point's y-coordinate can be odd or even, needs recording for complete reconstruction**2. Precise Order of Witness Data****Order Importance**:- Bitcoin Core parses witness data in fixed order- Position `[n-1]`: Control block (last element)- Position `[n-2]`: Script code (second to last)- Positions `[0...n-3]`: Input parameters for script execution**3. Hexadecimal Encoding of Preimage****Encoding Principle**:- Bitcoin Script processes hexadecimal byte data- Strings must first be encoded to UTF-8 bytes, then converted to hexadecimal- This ensures OP_SHA256 in script can correctly process input data### Actual Transaction Execution Result AnalysisAfter running the above code, we can observe real transactions on testnet:**Transaction ID**: `68f7c8f0ab6b3c6f7eb037e36051ea3893b668c26ea6e52094ba01a7722e604f`**Witness Data Analysis**:```bashWitness Stack:[0] 68656c6c6f776f726c64                                                     # Preimage[1] a820936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af8851   # Script[2] c150be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3     # Control block```Let's verify the correctness of this data:After verifying script legitimacy and preimage correctness, we also need: **Control Block Verification** → Prove script is in Merkle root, **Address Restoration Verification** → Verify legitimacy through tweak, and finally **Stack Execution** → Execute script logic. These are also Bitcoin Core's verification steps.### Control Block Verification - Prove Script is in Merkle Tree### Address Restoration Verification - Verify Legitimacy Through Tweak

If you encounter Script Path spending failures, follow this systematic debugging approach:### 🔍 Debug Flowchart: "Script Path Spending Failed? Try This"**Step 1: Check Witness Stack Order**```✅ Correct Order: [preimage, script, control_block]❌ Wrong: [control_block, script, preimage]❌ Wrong: [script, preimage, control_block]```**Step 2: Verify Script Consistency**- Does reveal phase script EXACTLY match commit phase script?- Are all parameters (hash values, opcodes) identical?- Use the same `build_hash_lock_script()` function for both phases**Step 3: Validate Control Block**- Is internal pubkey correct?- Is parity flag (`is_odd`) set properly? → Get from `taproot_address.is_odd()`- Does script index match tree position? (0 for single leaf)**Step 4: Check Input Data Encoding**- Is preimage properly UTF-8 encoded then hex converted?- Example: `"helloworld" → bytes → "68656c6c6f776f726c64"`**Step 5: Address Restoration Test**- Can you rebuild the same Taproot address from internal key + script tree?- Do tweak calculations match between commit and reveal phases?### 1. Witness Data Order Errors**Quick Fix**: Always remember "Data → Code → Proof" order for Script Path witnesses.### 2. Script Serialization Issues### 3. Control Block Parity Errors### 4. Commit-Reveal Inconsistency

Next, let's observe the complete stack execution process of Hash Lock script:**Executing Script**: `OP_SHA256 OP_PUSHBYTES_32 936a185c...07af OP_EQUALVERIFY OP_PUSHNUM_1`### 0. Initial Stack State: Load Script Input```| 68656c6c6f776f726c64                             || (preimage_hex: "helloworld")                     |└──────────────────────────────────────────────────┘```### 1. OP_SHA256: Calculate SHA256 Hash of Stack Top ElementOP_SHA256 pops the preimage data from stack top, calculates its SHA256 hash, then pushes result back to stack:```| 936a185caaa266bb9cbe981e9e05cb78cd732b0...                      || b3280eb944412bb6f8f8f07af                                       ||  (computed_hash)                                                |└─────────────────────────────────────────────────────────────────┘```**(Calculation Process: SHA256("helloworld") = 936a185c...07af)**### 2. PUSH 32 bytes: Push Expected Hash ValueScript pushes preset expected hash value to stack top:```| 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af        |(expected_hash)                                                           || 936a185caaa266bb9cbe981e9e05cb78cd732b0b3280eb944412bb6f8f8f07af        | (computed_hash)                                                           |└─────────────────────────────────────────────────────────────────────────┘```**(Two identical hash values now in stack)**### 3. OP_EQUALVERIFY: Verify Hash Values EqualOP_EQUALVERIFY pops top two elements for comparison, continues execution if equal, otherwise script fails:```| (empty_stack) |└───────────────┘```**(Verification Success: 936a185c...07af == 936a185c...07af, both elements consumed)**### 4. OP_TRUE: Push Success FlagFinally, OP_TRUE (OP_PUSHNUM_1) pushes value 1 to stack, marking successful script execution:```| 01 (true_value) |└─────────────────┘```**(Script Execution Success: Stack top is non-zero value)**

Through actual code implementation and on-chain data analysis, we can clearly see the differences between the two spending methods:| Aspect | Key Path | Script Path || --- | --- | --- || **Witness Data** | 1 element (64-byte signature) | 3 elements (input+script+control block) || **Transaction Size** | ~153 bytes | ~234 bytes || **Privacy Level** | Complete privacy, zero information leakage | Partial privacy, only exposes used script branch || **Verification Complexity** | Single Schnorr signature verification | Control block verification + script execution || **Fee Cost** | Lowest cost | Medium cost (~50% additional overhead) |This **selective reveal** design enables Taproot to support various complex application scenarios: digital goods sales, bounty tasks, conditional escrow, multi-party contracts, etc., while maintaining maximum privacy when unused.

Unlike P2SH, where **all conditions are revealed** when spending, Taproot Script Path ensures only the executed branch is ever seen. This fundamental shift redefines Bitcoin's privacy model for contracts.**Traditional Script Limitations:**- All spending conditions visible on-chain- Complex contracts easily identifiable by observers- Privacy compromised even for unused branches**Taproot's Privacy Innovation:**- Unused conditions remain hidden forever- Complex contracts indistinguishable from simple payments- Only executed logic revealed, preserving maximum privacyThis privacy-first design makes Taproot the foundation for Bitcoin's next generation of smart contracts, where complexity doesn't compromise confidentiality.

Through Alice's Hash Lock contract case, we deeply understand Taproot's revolutionary approach to Bitcoin smart contracts:### Power of Commit-Reveal Pattern1. **Commit Phase**: Commit complex conditional logic to an ordinary Taproot address, generate intermediate address to lock funds2. **Reveal Phase**: Choose Key Path or Script Path spending based on actual needs, only expose necessary information### Technical Implementation Mastery1. **Single Leaf Script Tree**: TapLeaf hash directly serves as Merkle root, no additional Merkle calculation needed2. **Control Block Verification**: Cryptographically prove script legitimacy through address restoration3. **Stack Execution**: Hash Lock implements conditional spending through hash matching verification### Development Best Practices1. **Tagged Hash Understanding**: Master hash tags for different purposes, ensure security2. **Witness Data Order**: Strictly follow [input parameters, script, control block] order3. **Systematic Debugging**: Use our debug flowchart to troubleshoot common issues4. **Code Consistency**: Use helper functions to ensure commit-reveal phase alignment### The Bigger PictureThis chapter establishes more than just Hash Lock implementation—it demonstrates Taproot's fundamental privacy revolution. By allowing complex contracts to masquerade as simple payments until execution, Taproot transforms Bitcoin's capability without sacrificing user privacy.**Next Steps**: In the following chapter, we'll explore **dual-leaf script trees**, learning how to organize multiple different spending conditions in one Taproot address, introducing real Merkle tree calculations, and experiencing the complete power of Taproot's script tree architecture for more complex application scenarios.